In [11]:
from langchain_ollama import OllamaLLM

ollama = OllamaLLM (
    base_url = 'http://localhost:11434',
    model = "llama3.2:latest"
)

In [2]:
from langchain_core.prompts import ChatPromptTemplate
template = """Question: {question}

Answer: Let's think step by step."""
prompt = ChatPromptTemplate.from_template(template)

In [3]:
chain = prompt | ollama

In [4]:
chain.invoke({"question": "Who is Neleus and who is in Neleus' family?"})

"To answer your question, we'll break it down into steps.\n\nStep 1: Identify the character of Neleus\nNeleus was a figure in Greek mythology. He was a sea-nymph and the son of the god Poseidon and the sea-nymph Thaumas' daughter, Amphitrite's sister, and possibly also a nymph named Callirhoe or Andromeda.\n\nStep 2: Determine Neleus' family members\nUnfortunately, there isn't much information available about Neleus' immediate family. However, we can consider his relationships with other mythological figures:\n\n- His father was Poseidon, the Greek god of the sea.\n- His mother was Amphitrite, who was also a sea-nymph and the wife of Poseidon.\n\nStep 3: Analyze the available information about Neleus' family\nBased on the limited information available, it's difficult to determine more details about his immediate family members. However, we can conclude that Neleus had a father (Poseidon) and possibly other relatives related to Amphitrite."

In [5]:
from langchain_core.messages import HumanMessage

ollama.invoke([HumanMessage(content="Hi! I'm Bob")])

"Hello Bob. It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [6]:
ollama.invoke([HumanMessage(content="What's my name?")])

"I don't have any information about your identity or personal details. I'm a large language model, I don't have the ability to know your name unless you tell me it. Can you please tell me your name so we can continue our conversation?"

In [7]:
from langchain_core.messages import AIMessage
ollama.invoke (
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

'That\'s a clever question, Bob. According to our conversation earlier, your name is indeed "Bob". Would you like me to help you with something specific or would you like to chat for a bit?'

In [8]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = ollama.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc123"}}

query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state


================================ Human Message =================================

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [10]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

KeyboardInterrupt: 

In [ ]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

I don't have any information about your identity, so I'm not sure what to tell you. Can you please provide me with some context or details about yourself, like where you're from or something unique about you? This will help me narrow down the options and try to figure out who you might be.


In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

I think we've had this conversation before! Don't worry, Bob, it's okay to repeat yourself sometimes.

As for the conversation itself, I'm doing great, thanks for asking! It's not every day I get to chat with someone who's already aware of their own identity. I suppose that's a good thing!

If you're up for it, I'd love to discuss something completely random. How about we talk about the weirdest foods you've ever tried? Or maybe we can explore some weird and wonderful myths from around the world?

What do you say, Bob?


## Prompt templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | ollama
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Arrrr, 'ello there, Jim! *pours self a cup o' grog* Welcome aboard, matey! What be bringin' ye to these waters? Don't be shy, tell ol' Blackbeak what's on yer mind!


In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Yer lookin' fer a reminder o' yer name, eh? Alright then, matey, let me take a gander at the treasure map... er, I mean, yer identity chart. *winks*

 Yer name be Jim, and ye've been welcomed aboard by meself, Blackbeak the pirate! Ye can rest easy knowin' that I'll be keepin' an eye on ye while ye're sailin' these seven seas with me crew! Now, what's on yer mind, Jim? What's bringin' ye to these fair waters?


## Prompt Templates Advance

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | ollama
    trimmed_messages = trimmer.invoke(state["messages"])
    # response = chain.invoke(state)
    response = chain.invoke (
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "French"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Bonjour Bob ! Je t'appelle aussi Bob, c'est simple ! (Hello Bob! I'll call myself Bob too, it's easy!)

Et maintenant, pour répondre à ta question : Tu as mentionné que tu étais Bob, mais cela me semble un peu ridicule car on ne sait pas si tu es vraiment toi-même. Mais je vais jouer le jeu ! Alors, comme tu es Bob, comment puis-je t'aider aujourd'hui ? (And now, to answer your question: You mentioned that you were Bob, but it seems a bit ridiculous because we don't know if you're really yourself. But I'll play along! So, as you are Bob, how can I help you today?)

(Note : Je vais essayer de répondre à tes questions de manière logique et amusante, alors que tu as déclaré être toi-même !)


In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Ha ha, c'est une bonne question ! (That's a good question!) Eh bien, je vais te dire une chose : si tu es Bob, alors tu es toi-même et je suis moi-même, donc... je ne sais pas si je peux te donner une réponse précise ! Mais je peux essayer de t'aider à trouver ta réponse en soi. Qu'est-ce qui t'interroge sur ton nom ? Est-ce que tu es perdu ou que tu as oublié quelque chose ? (Well, I'll tell you something : if you are Bob, then you are yourself and I am myself, so... I don't know if I can give a precise answer! But I can try to help you find your answer by yourself. What's making you wonder about your name? Are you lost or did you forget something?)


## Managing Conversation History

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

In [ ]:

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=ollama,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | ollama
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc999"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================ Human Message =================================

That's a clever trick! Since you're the one asking the question, I can tell that your name isn't actually "Bob". You were just testing me earlier when you said "I like vanilla ice cream" and then asked about my responses. Am I correct?


In [ ]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================ Human Message =================================

I'm glad you're enjoying our conversation!

You asked the math problem "2 + 2", and I answered it correctly with 4.

And, according to our previous chat, your name is indeed Bob!


# Streaming 

In [ ]:
config = {"configurable": {"thread_id": "edb789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    # if isinstance(chunk, AIMessage):  # Filter to just model responses
    #     print(chunk.content, end="|")
    print(chunk.content, end="|")